<a href="https://colab.research.google.com/github/lightweightcobra/Deepfake-Creation/blob/main/Gradient_Descent_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt


In [4]:
df = pd.read_csv("/content/insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']], df.bought_insurance, test_size = 0.2, random_state = 15)

In [ ]:
len(X_train)

In [7]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] /= 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] /= 100

In [ ]:
X_train_scaled

In [9]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape = (2,), activation = 'sigmoid', kernel_initializer = 'ones', bias_initializer = 'zeros')
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train_scaled, y_train, epochs = 5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 13ms/step - loss: 0.5057 - accuracy: 0.7273
Epoch 2502/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5057 - accuracy: 0.7273
Epoch 2503/5000
1/1 [==============================] - 0s 13ms/step - loss: 0.5056 - accuracy: 0.7273
Epoch 2504/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5056 - accuracy: 0.7273
Epoch 2505/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5056 - accuracy: 0.7273
Epoch 2506/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5055 - accuracy: 0.7273
Epoch 2507/5000
1/1 [==============================] - 0s 13ms/step - loss: 0.5055 - accuracy: 0.7273
Epoch 2508/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5054 - accuracy: 0.7273
Epoch 2509/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5054 - accuracy: 0.7273
Epoch 2510/5000
1/1 [===========================

In [10]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 155ms/step - loss: 0.4291 - accuracy: 0.8333


[0.42906585335731506, 0.8333333134651184]

In [11]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 73ms/step


array([[0.36212626],
       [0.8114617 ],
       [0.73289955],
       [0.11885991],
       [0.15900403],
       [0.74376935]], dtype=float32)

In [17]:
X_test_scaled

,age,affordibility
19,0.18,1
25,0.54,1
27,0.46,1
18,0.19,0
1,0.25,0
2,0.47,1


In [14]:
coeff, intercept = model.get_weights()
coeff, intercept

(array([[5.626898 ],
        [1.4933927]], dtype=float32),
 array([-3.0723817], dtype=float32))

In [15]:
def sigmoid(x):
  return 1/ (1+ np.exp(-x))

In [16]:
def prediction_function(age, afford):
  predict = coeff[0]* age + coeff[1]*afford + intercept
  return sigmoid(predict)

In [18]:
print("The prediction will be: ", prediction_function(0.54, 1))

The prediction will be:  [0.8114616]


In [19]:
def log_loss(y_true, y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i, epsilon) for i in y_predicted]
  y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true)* np.log(1-y_predicted_new))


In [25]:
def gradient_descent(age, afford, y_true, epoch, loss_threshold):
  w1 = w2 = 1
  bias = 0
  rate = 0.5
  n = len(age)

  for i in range(epoch):
    weighted_sum = w1*age + w2*afford + bias
    y_predicted = sigmoid(weighted_sum)

    loss = log_loss(y_true, y_predicted)

    w1d = (1/n)*np.dot(age.T, (y_predicted-y_true))
    w2d = (1/n)*np.dot(afford.T, (y_predicted-y_true))

    bias_d = np.mean(y_predicted - y_true)

    w1 = w1 - rate*w1d
    w2 = w2 - rate*w2d
    bias = bias - rate*bias_d

    print(f'Epochs:{i}, w1: {w1}, w2:{w2}, bias:{bias}, loss:{loss}')

    if loss <= loss_threshold: break
  return w1, w2, bias



In [27]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train, 1000, 0.4306)

Epochs:0, w1: 0.9828250739733334, w2:0.9529630791349588, bias:-0.09641968209539553, loss:0.6777364328975026
Epochs:1, w1: 0.9706611956342757, w2:0.9143272166100153, bias:-0.1804741192262762, loss:0.6557817945661433
Epochs:2, w1: 0.9630325691651972, w2:0.8833946271097505, bias:-0.2533631969166629, loss:0.6396496230261026
Epochs:3, w1: 0.9594047979359941, w2:0.8593154509942684, bias:-0.31641595204712114, loss:0.6279467455083667
Epochs:4, w1: 0.9592331459312166, w2:0.8411837855333407, bias:-0.3709749554008529, loss:0.6194971932197469
Epochs:5, w1: 0.9619974280663424, w2:0.8281104477501106, bias:-0.41831415321744003, loss:0.6133704781458633
Epochs:6, w1: 0.9672232211202912, w2:0.8192698127239382, bias:-0.4595900177845542, loss:0.6088659704748213
Epochs:7, w1: 0.9744919127657256, w2:0.8139245508003335, bias:-0.4958194107254899, loss:0.6054756880088077
Epochs:8, w1: 0.9834429202059429, w2:0.8114343322975994, bias:-0.5278759108971158, loss:0.6028417200281034
Epochs:9, w1: 0.9937710810255609, 

(5.631608838182138, 1.5059888872235303, -3.093619669248222)